https://www.kaggle.com/datasets/tourist55/alzheimers-dataset-4-class-of-images

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from matplotlib import pyplot as plt

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
images=tf.data.Dataset.list_files('Alzheimer_s Dataset/test/NonDemented/*.jpg',shuffle=False)

def load_img(x):
    byte=tf.io.read_file(x)
    img=tf.io.decode_jpeg(byte,channels=3)
    return img

images=images.map(load_img)
images.as_numpy_iterator().next().shape

In [ ]:
def scale_images(data): 
    image = data['image']
    return image / 255

images=tf.data.Dataset.list_files('Alzheimer_s Dataset/test/NonDemented/*.jpg',shuffle=False)
images=images.map(load_img)

images = images.map(lambda x: tf.image.resize(x, (112,112)))
images = images.map(lambda x: x/255)
images = images.cache()

# Batch into 128 images per sample
images = images.batch(32)
# Reduces the likelihood of bottlenecking 
images = images.prefetch(64)

In [ ]:
images.as_numpy_iterator().next().shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D

def build_generator(): 
    model = Sequential([
        # Takes in random values and reshapes it to 7x7x128
        # Beginnings of a generated image
        Dense(28*28*128, input_dim=128),
        LeakyReLU(0.2),
        Reshape((28,28,128)),

        UpSampling2D(),
        Conv2D(256, 5, padding='same'),
        LeakyReLU(0.2),

        UpSampling2D(),
        Conv2D(256, 5, padding='same'),
        LeakyReLU(0.2),

        Conv2D(128, 4, padding='same'),
        LeakyReLU(0.2),
        
        Conv2D(64, 4, padding='same'),
        LeakyReLU(0.2),
        
        # Conv layer to get to one channel
        Conv2D(3, 4, padding='same', activation='sigmoid')])
    
    return model

In [ ]:
generator = build_generator()
generator.summary()

In [ ]:
def build_discriminator(): 
    model = Sequential([
        Conv2D(32, 5, input_shape = (112,112,3)),
        LeakyReLU(0.2),
        Dropout(0.4),

        Conv2D(64, 5),
        LeakyReLU(0.2),
        Dropout(0.4),

        Conv2D(128, 5),
        LeakyReLU(0.2),
        Dropout(0.4),

        Conv2D(256, 5),
        LeakyReLU(0.2),
        Dropout(0.4),
        
        Flatten(),
        Dropout(0.4),
        Dense(1, activation='sigmoid')])
    
    return model 

In [ ]:
discriminator = build_discriminator()
discriminator.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model

g_opt = Adam(learning_rate=0.0001) 
d_opt = Adam(learning_rate=0.00001) 
g_loss = BinaryCrossentropy()
d_loss = BinaryCrossentropy()

In [ ]:
class FashionGAN(Model): 
    def __init__(self, generator, discriminator, *args, **kwargs):
        # Pass through args and kwargs to base class 
        super().__init__(*args, **kwargs)
        
        # Create attributes for gen and disc
        self.generator = generator 
        self.discriminator = discriminator 
        
    def compile(self, g_opt, d_opt, g_loss, d_loss, *args, **kwargs): 
        # Compile with base class
        super().compile(*args, **kwargs)
        
        # Create attributes for losses and optimizers
        self.g_opt = g_opt
        self.d_opt = d_opt
        self.g_loss = g_loss
        self.d_loss = d_loss 

    def train_step(self, batch):
        # Get the data 
        real_images = batch
        fake_images = self.generator(tf.random.normal((128,128,1)), training=False)
        
        # Train the discriminator
        with tf.GradientTape() as d_tape: 
            # Pass the real and fake images to the discriminator model
            yhat_real = self.discriminator(real_images, training=True) 
            yhat_fake = self.discriminator(fake_images, training=True)
            yhat_realfake = tf.concat([yhat_real, yhat_fake], axis=0)
            
            # Create labels for real and fakes images
            y_realfake = tf.concat([tf.zeros_like(yhat_real), tf.ones_like(yhat_fake)], axis=0)
            
            # Add some noise to the TRUE outputs
            noise_real = 0.15*tf.random.uniform(tf.shape(yhat_real))
            noise_fake = -0.15*tf.random.uniform(tf.shape(yhat_fake))
            y_realfake += tf.concat([noise_real, noise_fake], axis=0)
            
            # Calculate loss - BINARYCROSS 
            total_d_loss = self.d_loss(y_realfake, yhat_realfake)
            
        # Apply backpropagation - nn learn 
        dgrad = d_tape.gradient(total_d_loss, self.discriminator.trainable_variables) 
        self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables))
        
        # Train the generator 
        with tf.GradientTape() as g_tape: 
            # Generate some new images
            gen_images = self.generator(tf.random.normal((128,128,1)), training=True)
                                        
            # Create the predicted labels
            predicted_labels = self.discriminator(gen_images, training=False)
                                        
            # Calculate loss - trick to training to fake out the discriminator
            total_g_loss = self.g_loss(tf.zeros_like(predicted_labels), predicted_labels) 
            
        # Apply backprop
        ggrad = g_tape.gradient(total_g_loss, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables))
        
        return {"d_loss":total_d_loss, "g_loss":total_g_loss}

In [ ]:
# Create instance of subclassed model
fashgan = FashionGAN(generator, discriminator)

# Compile the model
fashgan.compile(g_opt, d_opt, g_loss, d_loss)

In [ ]:
import os
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import Callback

In [ ]:
class ModelMonitor(Callback):
    def __init__(self, num_img=128, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.uniform((self.num_img, self.latent_dim,1))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = array_to_img(generated_images[i])
            img.save(os.path.join('images', f'generated_img_{epoch}_{i}.png'))

In [ ]:
# Recommend 2000 epochs
hist = fashgan.fit(images, epochs=1, callbacks=[ModelMonitor()])